In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt


df = pd.read_csv('data_downsampling.csv', header=None)
df_y = pd.read_csv('data_output.csv', header=None)

# 첫 번째 열을 레이블로 가정하고 나머지를 특성으로 사용
X = df.iloc[:,:].values  # 모든 행과 1번째 열 이후의 데이터
y = df_y.iloc[:, 0].values  # 첫 번째 열 데이터 (레이블)

# 레이블 인코딩
labels, y_encoded = np.unique(y, return_inverse=True)
y_encoded = to_categorical(y_encoded)  # 원-핫 인코딩

# 데이터 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 형태 변환: (샘플 수, 시퀀스 길이, 특성 수)
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_encoded, test_size=0.2, random_state=42)

# 5. CNN 모델 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

# 6. 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 7. 모델 학습
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

# 8. 학습 손실(loss) 그래프 그리기
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 9. 예측 수행
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# 10. 각 클래스별 정확도 출력
report = classification_report(y_test_classes, y_pred_classes, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# Confusion matrix for better understanding
cm = confusion_matrix(y_test_classes, y_pred_classes)

# Display classification report
print(df_report)

# Optional: Save the report to a CSV file
df_report.to_csv('classification_report.csv', index=True)
